In [1]:
import tensorflow as tf
print("Num GPUs Available: ", tf.config.experimental.list_physical_devices('GPU'))

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [22]:
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline

In [3]:
import sys
import numpy as np
from tensorflow.keras.layers import Dense, Reshape
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

block_size = 128
num_blocks=4
OPTMIZER = Adam(lr=0.0002, decay=8e-9)

generator = Sequential()
generator.add(Dense(block_size, input_shape=(100, )))
generator.add(LeakyReLU(alpha=0.02))
generator.add(BatchNormalization(momentum=0.8))

for i in range(num_blocks-1):
    block_size = block_size * 2
    generator.add(Dense(block_size))
    generator.add(LeakyReLU(alpha=0.2))
    generator.add(BatchNormalization(momentum=0.8))

generator.add(Dense(28 * 28 * 1, activation='tanh'))
generator.add(Reshape((28, 28, 1)))
generator.compile(loss="binary_crossentropy", optimizer=OPTMIZER)
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               12928     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128)               0         
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                        

C:\Users\rohit\anaconda3\envs\gpuEnv\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
import sys
import numpy as np
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

SHAPE =  (28, 28, 1)
CAPACITY = 28 * 28 * 1
OPTMIZER = Adam(lr=0.0002, decay=8e-9)

discriminator = Sequential()
discriminator.add(Flatten(input_shape = SHAPE))
discriminator.add(Dense(CAPACITY, input_shape=SHAPE))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(int(CAPACITY/2)))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss="binary_crossentropy", optimizer=OPTMIZER)
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_5 (Dense)             (None, 784)               615440    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 392)               307720    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 392)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 393       
                                                                 
Total params: 923,553
Trainable params: 923,553
Non-tr

In [5]:
OPTIMIZER = Adam(lr=0.0002, decay=8e-9)
discriminator.trainable = False

gan = Sequential()
gan.add(generator)
gan.add(discriminator)
gan.compile(loss="binary_crossentropy", optimizer=OPTMIZER)
gan.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 28, 28, 1)         1514128   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 923553    
                                                                 
Total params: 2,437,681
Trainable params: 1,510,288
Non-trainable params: 927,393
_________________________________________________________________


In [23]:
 def load_MNIST(model_type=-1):
    allowed_types = [-1,0,1,2,3,4,5,6,7,8,9]
    if model_type not in allowed_types:
        print('ERROR: Only Integer Values from -1 to 9 are allowed')

    (X_train, Y_train), (_, _) = mnist.load_data()
    if model_type!=-1:
        X_train = X_train[np.where(Y_train==int(model_type))[0]]
    
    # Rescale -1 to 1
    # Find Normalize Function from CV Class  
    X_train = ( np.float32(X_train) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    return X_train

In [25]:
X_train.shape

(60000, 28, 28, 1)

In [7]:
W = 28
H = 28
C = 1
EPOCHS = 50000
BATCH = 32
CHECKPOINT = 500
model_type=-1
LATENT_SPACE_SIZE = 100

# generator = Generator(height=H, width=W, channels=C, latent_size=LATENT_SPACE_SIZE)
# discriminator = Discriminator(height=H, width=W, channels=C)
# gan = GAN(generator=generator.Generator, discriminator=discriminator.Discriminator)

X_train = load_MNIST()



def train():
    

    for e in range(EPOCHS):
        # Train Discriminator
        # Make the training batch for this model be half real, half noise
        # Grab Real Images for this training batch
        count_real_images = int(BATCH/2)
        starting_index = randint(0, (len(X_train)-count_real_images))
        real_images_raw = X_train[ starting_index : (starting_index + count_real_images) ]
        x_real_images = real_images_raw.reshape( count_real_images, W, H, C )
        y_real_labels = np.ones([count_real_images,1])

        # Grab Generated Images for this training batch
        latent_space_samples = sample_latent_space(count_real_images)
        x_generated_images = generator.predict(latent_space_samples)
        y_generated_labels = np.zeros([BATCH-count_real_images,1])

        # Combine to train on the discriminator
        x_batch = np.concatenate( [x_real_images, x_generated_images] )
        y_batch = np.concatenate( [y_real_labels, y_generated_labels] )

        # Now, train the discriminator with this batch
        discriminator_loss = discriminator.train_on_batch(x_batch,y_batch)

        # Generate Noise
        x_latent_space_samples = sample_latent_space(BATCH)
        y_generated_labels = np.ones([BATCH,1])
        generator_loss = gan.train_on_batch(x_latent_space_samples,y_generated_labels)

        

        if e % CHECKPOINT == 0 :
            print ('Epoch: '+str(int(e))+', [Discriminator :: Loss: '+str(discriminator_loss)+'], [ Generator :: Loss: '+str(generator_loss)+']')
            plot_checkpoint(e)
    return

In [8]:
def sample_latent_space(instances):
    return np.random.normal(0, 1, (instances, LATENT_SPACE_SIZE))

def plot_checkpoint(e):
    filename = str(e)+".png"
    gfilename = str(e)+".png"

    noise = sample_latent_space(16)
    images = generator.predict(noise)
    
    plt.figure(figsize=(10,10))
    for i in range(images.shape[0]):
        plt.subplot(4, 4, i+1)
        image = images[i, :, :, :]
        image = np.reshape(image, [H,W])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(filename)
    plt.savefig(gfilename)
    plt.close('all')
    return

In [9]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
with tf.device('/GPU:0'):
    train()

1/1 [==============================] - 1s 758ms/step
Epoch: 0, [Discriminator :: Loss: 0.6310312747955322], [ Generator :: Loss: 0.6492060422897339]
1/1 [==============================] - 0s 16ms/step
Epoch: 500, [Discriminator :: Loss: 0.21208226680755615], [ Generator :: Loss: 7.060812950134277]
1/1 [==============================] - 0s 16ms/step
Epoch: 1000, [Discriminator :: Loss: 0.04153551533818245], [ Generator :: Loss: 6.8815813064575195]
1/1 [==============================] - 0s 31ms/step
Epoch: 1500, [Discriminator :: Loss: 0.011914667673408985], [ Generator :: Loss: 7.30399227142334]
1/1 [==============================] - 0s 16ms/step
Epoch: 2000, [Discriminator :: Loss: 0.020614316686987877], [ Generator :: Loss: 7.7010040283203125]
1/1 [==============================] - 0s 31ms/step
Epoch: 2500, [Discriminator :: Loss: 0.11688310652971268], [ Generator :: Loss: 1.896493673324585]
1/1 [==============================] - 0s 33ms/step
Epoch: 3000, [Discriminator :: Loss: 0.137

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1/1 [==============================] - 0s 56ms/step
Epoch: 33500, [Discriminator :: Loss: 0.5725645422935486], [ Generator :: Loss: 2.2249107360839844]
1/1 [==============================] - 0s 64ms/step
Epoch: 34000, [Discriminator :: Loss: 0.4156651496887207], [ Generator :: Loss: 1.533139944076538]
1/1 [==============================] - 0s 37ms/step
Epoch: 34500, [Discriminator :: Loss: 0.7213178277015686], [ Generator :: Loss: 1.8496129512786865]
1/1 [==============================] - ETA: 0s

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1/1 [==============================] - 0s 47ms/step
Epoch: 35500, [Discriminator :: Loss: 0.3620286285877228], [ Generator :: Loss: 2.1304469108581543]
1/1 [==============================] - 0s 62ms/step
Epoch: 36000, [Discriminator :: Loss: 0.5491209626197815], [ Generator :: Loss: 1.5214402675628662]
1/1 [==============================] - 0s 47ms/step
Epoch: 36500, [Discriminator :: Loss: 0.3976130485534668], [ Generator :: Loss: 2.181232452392578]
1/1 [==============================] - 0s 46ms/step
Epoch: 37000, [Discriminator :: Loss: 0.46008673310279846], [ Generator :: Loss: 2.6188058853149414]
1/1 [==============================] - 0s 47ms/step
Epoch: 37500, [Discriminator :: Loss: 0.24759769439697266], [ Generator :: Loss: 1.9021531343460083]
1/1 [==============================] - 0s 63ms/step
Epoch: 38000, [Discriminator :: Loss: 0.3339543342590332], [ Generator :: Loss: 1.8226876258850098]
1/1 [==============================] - 0s 47ms/step
Epoch: 38500, [Discriminator :: Los

In [16]:
# Save the generator model
generator.save('gan_generatorv4.h5')

# Save the discriminator model
discriminator.save('gan_discriminatorv4.h5')

# Save the GAN model (combined model)
gan.save('gan_combinedv4.h5')


In [17]:
from tensorflow.keras.models import load_model

# Load the generator model
generator = load_model('gan_generatorv4.h5')

# Load the discriminator model
discriminator = load_model('gan_discriminatorv4.h5')

# Load the GAN model (combined model)
gan = load_model('gan_combinedv4.h5')


In [21]:
def sample_latent_spac(instances, input_number):
    return np.full((instances, LATENT_SPACE_SIZE), input_number)

def generate_new_image(input_number):
    filename = str(input_number)+".png"
    gfilename = str(input_number)+".png"

    noise = sample_latent_spac(16, input_number)
    images = generator.predict(noise)
    
    plt.figure(figsize=(10,10))
    for i in range(images.shape[0]):
        plt.subplot(4, 4, i+1)
        image = images[i, :, :, :]
        image = np.reshape(image, [H,W])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(filename)
    plt.savefig(gfilename)
    plt.close('all')
    return
generate_new_image( 2)

1/1 [==============================] - 0s 160ms/step
